In [1]:
%cd "OneDrive - National University of Singapore"

C:\Users\wongp\OneDrive - National University of Singapore


In [2]:
%cd CS4248/Project/db/

C:\Users\wongp\OneDrive - National University of Singapore\CS4248\Project\db


In [3]:
import os
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from collections import Counter
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import nltk
from nltk.stem import WordNetLemmatizer

Convert training data into csv file

In [ ]:
import os
import pandas as pd

pathNeg = "train/neg"
files = os.listdir(pathNeg)
data = []
for file in files:
  if os.path.isfile(os.path.join(pathNeg, file)):
    id = int(file.split("_")[0])
    with open(os.path.join(pathNeg, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 0])

sizeOfNeg = 12500

pathPos = "train/pos"
files = os.listdir(pathPos)
for file in files:
  if os.path.isfile(os.path.join(pathPos, file)):
    id = int(file.split("_")[0]) + sizeOfNeg
    with open(os.path.join(pathPos, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 1])


df = pd.DataFrame(data, columns = ['id', 'text', 'score']).set_index('id').sort_index()
df.to_csv('train.csv')

Convert testing data into csv file

In [ ]:
pathNeg = "test/neg"
files = os.listdir(pathNeg)
data = []
for file in files:
  if os.path.isfile(os.path.join(pathNeg, file)):
    id = int(file.split("_")[0])
    with open(os.path.join(pathNeg, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 0])

sizeOfNeg = 12500

pathPos = "test/pos"
files = os.listdir(pathPos)
for file in files:
  if os.path.isfile(os.path.join(pathPos, file)):
    id = int(file.split("_")[0]) + sizeOfNeg
    with open(os.path.join(pathPos, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 1])


df = pd.DataFrame(data, columns = ['id', 'text', 'score']).set_index('id').sort_index()
df.to_csv('test.csv')

convert vocab into csv file

In [ ]:
with open("imdb.vocab", encoding='utf8') as f:
  vocab = []
  for line in f:
    vocab.append(line.rstrip())

vocab_df = pd.DataFrame(vocab, columns=['vocab'])
print(vocab_df)
vocab_df.to_csv('vocab_given.csv', index=False)

                   vocab
0                    the
1                    and
2                      a
3                     of
4                     to
...                  ...
89522          copywrite
89523             artbox
89524          kinky-sex
89525           urrrghhh
89526  investigator-like

[89527 rows x 1 columns]


In [4]:
from nltk.tokenize import word_tokenize

def tokenizeReview(review):
  review = review.replace('.', ' . ')
  tokens = word_tokenize(review.lower())
  return tokens

Get vocab from training set

In [ ]:
from collections import Counter
reviews = train_df['text'].tolist()
reviews = list(map(tokenizeReview, reviews))
reviews = [item for review in reviews for item in review]
vdict = Counter(reviews).most_common()

In [ ]:
vocabs = []
for pair in vdict:
  vocabs.append(pair[0])

vocab_df = pd.DataFrame(vocabs, columns=['vocab'])
vocab_df.to_csv('vocab.csv', index=False)
print(vocab_df)

              vocab
0               the
1                 .
2                 ,
3               and
4                 a
...             ...
99203    people-and
99204         padre
99205   unmatchable
99206  scenery-wise
99207  non-teenager

[99208 rows x 1 columns]


Read training data and testing data from csv file (START HERE after initialization)

In [5]:
train_df = pd.read_csv('train/train.csv')
test_df = pd.read_csv('test/test.csv')
vocab_df = pd.read_csv('vocab.csv')
print(vocab_df)

              vocab
0               the
1                 .
2                 ,
3               and
4                 a
...             ...
99203    people-and
99204         padre
99205   unmatchable
99206  scenery-wise
99207  non-teenager

[99208 rows x 1 columns]


In [ ]:
# vocab_dict = pd.Series(vocab_df.index + 1, index=vocab_df.vocab).to_dict()
# print(vocab_dict)

In [6]:
def convertToIndex(review):
  vec = []
  for word in review:
    if word in vocab_dict:
      vec.append(int(vocab_dict[word]))
    else:
      vec.append(0)
  return vec

In [7]:
train_df['tokens'] = train_df['text'].apply(tokenizeReview)

In [ ]:
# train_df['vec'] = train_df['tokens'].apply(convertToIndex)

In [8]:
train_df

,id,text,score,tokens
0,0,Story of a man who has unnatural feelings for ...,0,"[story, of, a, man, who, has, unnatural, feeli..."
1,1,Robert DeNiro plays the most unbelievably inte...,0,"[robert, deniro, plays, the, most, unbelievabl..."
2,2,"I saw the capsule comment said ""great acting.""...",0,"[i, saw, the, capsule, comment, said, ``, grea..."
3,3,If I had not read Pat Barker's 'Union Street' ...,0,"[if, i, had, not, read, pat, barker, 's, 'unio..."
4,4,This fanciful horror flick has Vincent Price p...,0,"[this, fanciful, horror, flick, has, vincent, ..."
...,...,...,...,...
24995,24995,What's Good About It: Some inventive and genui...,1,"[what, 's, good, about, it, :, some, inventive..."
24996,24996,For years we've been watching every horror fil...,1,"[for, years, we, 've, been, watching, every, h..."
24997,24997,If you haven't already seen this movie of Mary...,1,"[if, you, have, n't, already, seen, this, movi..."
24998,24998,this movie is the best movie ever it has a lot...,1,"[this, movie, is, the, best, movie, ever, it, ..."


In [ ]:
print('Maximum review length: {}'.format(
len(max((train_df['tokens']), key=len))))
print('Minimum review length: {}'.format(
len(min((train_df['tokens']), key=len))))

Maximum review length: 2820
Minimum review length: 11


Lemmatizer

In [9]:
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')

def lemmatizer(reviewTokens):
  punctuations="?:!.,;"
  for word in reviewTokens:
    if word in punctuations:
      reviewTokens.remove(word)
  reviewLemm = []
  for word in reviewTokens:
    newWord = wordnet_lemmatizer.lemmatize(word, pos="v")
    reviewLemm.append(newWord)

  return reviewLemm


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wongp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\wongp\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [10]:
train_df['lemmTokens'] = train_df['tokens'].apply(lemmatizer)

In [ ]:
train_df

,id,text,score,tokens,lemmTokens
0,0,Story of a man who has unnatural feelings for ...,0,"[story, of, a, man, who, has, unnatural, feeli...","[story, of, a, man, who, have, unnatural, feel..."
1,1,Robert DeNiro plays the most unbelievably inte...,0,"[robert, deniro, plays, the, most, unbelievabl...","[robert, deniro, play, the, most, unbelievably..."
2,2,"I saw the capsule comment said ""great acting.""...",0,"[i, saw, the, capsule, comment, said, ``, grea...","[i, saw, the, capsule, comment, say, ``, great..."
3,3,If I had not read Pat Barker's 'Union Street' ...,0,"[if, i, had, not, read, pat, barker, 's, 'unio...","[if, i, have, not, read, pat, barker, 's, 'uni..."
4,4,This fanciful horror flick has Vincent Price p...,0,"[this, fanciful, horror, flick, has, vincent, ...","[this, fanciful, horror, flick, have, vincent,..."
...,...,...,...,...,...
24995,24995,What's Good About It: Some inventive and genui...,1,"[what, 's, good, about, it, some, inventive, a...","[what, 's, good, about, it, some, inventive, a..."
24996,24996,For years we've been watching every horror fil...,1,"[for, years, we, 've, been, watching, every, h...","[for, years, we, 've, be, watch, every, horror..."
24997,24997,If you haven't already seen this movie of Mary...,1,"[if, you, have, n't, already, seen, this, movi...","[if, you, have, n't, already, see, this, movie..."
24998,24998,this movie is the best movie ever it has a lot...,1,"[this, movie, is, the, best, movie, ever, it, ...","[this, movie, be, the, best, movie, ever, it, ..."


In [11]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words())
stop_words = stop_words.union(["br", "<", ">", "(", ")", "/", "''", "'s", "``", "'", "'ve", "'m", "."])
def remove_stopwords(review):
  tokens_without_sw = [word for word in review if not word in stop_words]

  return tokens_without_sw

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wongp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
train_df['lemmTokens'] = train_df['lemmTokens'].apply(remove_stopwords)

In [ ]:
train_df

,id,text,score,tokens,lemmTokens
0,0,Story of a man who has unnatural feelings for ...,0,"[story, of, a, man, who, has, unnatural, feeli...","[story, unnatural, feel, pig, start, open, sce..."
1,1,Robert DeNiro plays the most unbelievably inte...,0,"[robert, deniro, plays, the, most, unbelievabl...","[robert, deniro, play, unbelievably, intellige..."
2,2,"I saw the capsule comment said ""great acting.""...",0,"[i, saw, the, capsule, comment, said, ``, grea...","[saw, capsule, comment, say, great, act, opini..."
3,3,If I had not read Pat Barker's 'Union Street' ...,0,"[if, i, had, not, read, pat, barker, 's, 'unio...","[read, pat, barker, 'union, street, see, film,..."
4,4,This fanciful horror flick has Vincent Price p...,0,"[this, fanciful, horror, flick, has, vincent, ...","[fanciful, horror, flick, vincent, price, play..."
...,...,...,...,...,...
24995,24995,What's Good About It: Some inventive and genui...,1,"[what, 's, good, about, it, some, inventive, a...","[good, inventive, genuinely, creepy, little, e..."
24996,24996,For years we've been watching every horror fil...,1,"[for, years, we, 've, been, watching, every, h...","[years, watch, every, horror, film, dull, holl..."
24997,24997,If you haven't already seen this movie of Mary...,1,"[if, you, have, n't, already, seen, this, movi...","[n't, already, see, movie, mary-kate, ashley, ..."
24998,24998,this movie is the best movie ever it has a lot...,1,"[this, movie, is, the, best, movie, ever, it, ...","[movie, best, movie, ever, lot, live, action, ..."


In [ ]:
from collections import Counter
reviewsLem = train_df['lemmTokens'].tolist()
# reviewsLem = list(map(tokenizeReview, reviewsLem))
reviewsLem = [item for review in reviewsLem for item in review]
vdictLem = Counter(reviewsLem).most_common()
#print(vdictLem)

vdictLemToIdx = {}
idx = 1
for pair in vdictLem:
  vdictLemToIdx[pair[0]] = idx
  idx = idx + 1
print(vdictLemToIdx)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
train_df['lemmTokensJoined'] = [' '.join(map(str, l)) for l in train_df['lemmTokens']]
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(train_df['lemmTokensJoined'])
tfidf_dict_temp = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
#print(tfidf_dict_temp)





In [16]:
tfidf_dict = {}
for word in vdictLemToIdx:
  if word in tfidf_dict_temp:
    tfidf_dict[word] = tfidf_dict_temp[word]
  else:
    tfidf_dict[word] = 1

#print(tfidf_dict)

In [17]:
def convertToIndex(review):
  vec = []
  for word in review:
    if word in vdictLemToIdx:
      vec.append(int(vdictLemToIdx[word]))
    else:
      vec.append(0)
  return vec

In [18]:
train_df['vecLem'] = train_df['lemmTokens'].apply(convertToIndex)

In [19]:
train_df

,id,text,score,tokens,lemmTokens,lemmTokensJoined,vecLem
0,0,Story of a man who has unnatural feelings for ...,0,"[story, of, a, man, who, has, unnatural, feeli...","[story, unnatural, feel, pig, start, open, sce...",story unnatural feel pig start open scene terr...,"[16, 6244, 52, 2798, 78, 229, 51, 1083, 358, 1..."
1,1,Robert DeNiro plays the most unbelievably inte...,0,"[robert, deniro, plays, the, most, unbelievabl...","[robert, deniro, play, unbelievably, intellige...",robert deniro play unbelievably intelligent il...,"[524, 4211, 32, 3185, 906, 7612, 8, 2, 18979, ..."
2,2,"I saw the capsule comment said ""great acting.""...",0,"[i, saw, the, capsule, comment, said, ``, grea...","[saw, capsule, comment, say, great, act, opini...",saw capsule comment say great act opinion two ...,"[120, 7762, 297, 23, 28, 31, 518, 41, 28, 66, ..."
3,3,If I had not read Pat Barker's 'Union Street' ...,0,"[if, i, had, not, read, pat, barker, 's, 'unio...","[read, pat, barker, 'union, street, see, film,...",read pat barker 'union street see film would l...,"[157, 2573, 6405, 27980, 746, 6, 1, 13, 5, 434..."
4,4,This fanciful horror flick has Vincent Price p...,0,"[this, fanciful, horror, flick, has, vincent, ...","[fanciful, horror, flick, vincent, price, play...",fanciful horror flick vincent price play mad m...,"[13143, 107, 300, 2682, 1392, 32, 986, 5297, 3..."
...,...,...,...,...,...,...,...
24995,24995,What's Good About It: Some inventive and genui...,1,"[what, 's, good, about, it, some, inventive, a...","[good, inventive, genuinely, creepy, little, e...",good inventive genuinely creepy little effect ...,"[9, 3672, 1708, 784, 44, 148, 7, 1823, 14, 464..."
24996,24996,For years we've been watching every horror fil...,1,"[for, years, we, 've, been, watching, every, h...","[years, watch, every, horror, film, dull, holl...",years watch every horror film dull hollywood r...,"[65, 11, 83, 107, 1, 614, 251, 8869, 5, 120, 1..."
24997,24997,If you haven't already seen this movie of Mary...,1,"[if, you, have, n't, already, seen, this, movi...","[n't, already, see, movie, mary-kate, ashley, ...",n't already see movie mary-kate ashley say wai...,"[3, 357, 6, 2, 11171, 3953, 23, 350, 155, 68, ..."
24998,24998,this movie is the best movie ever it has a lot...,1,"[this, movie, is, the, best, movie, ever, it, ...","[movie, best, movie, ever, lot, live, action, ...",movie best movie ever lot live action great ev...,"[2, 45, 2, 48, 60, 72, 103, 28, 191, 11, 183, ..."


In [20]:
max_words = 500
dataset = sequence.pad_sequences(train_df['vecLem'], maxlen=max_words)
y = train_df['score']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(dataset, y, test_size=0.3, stratify=y, random_state = 42)


In [21]:
from gensim.models import Word2Vec
import gensim.downloader as api

v2w_model = api.load('glove-wiki-gigaword-50')
print(v2w_model['computer'])

[ 0.079084 -0.81504   1.7901    0.91653   0.10797  -0.55628  -0.84427
 -1.4951    0.13418   0.63627   0.35146   0.25813  -0.55029   0.51056
  0.37409   0.12092  -1.6166    0.83653   0.14202  -0.52348   0.73453
  0.12207  -0.49079   0.32533   0.45306  -1.585    -0.63848  -1.0053
  0.10454  -0.42984   3.181    -0.62187   0.16819  -1.0139    0.064058
  0.57844  -0.4556    0.73783   0.37203  -0.57722   0.66441   0.055129
  0.037891  1.3275    0.30991   0.50697   1.2357    0.1274   -0.11434
  0.20709 ]


In [22]:
embedding_dim = 100
vocab_len = len(vdictLemToIdx) + 1
embed_vector_len = v2w_model['computer'].shape[0]
emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, i in vdictLemToIdx.items():
  if word in v2w_model:
    embedding_vector = v2w_model.get_vector(word)
    emb_matrix[i] = embedding_vector


In [ ]:
emb_matrix[1]

array([ 0.06912   ,  0.22159   , -0.66613001, -0.14060999,  0.063365  ,
        0.41747001, -0.21723001, -1.04009998,  0.026069  ,  1.33819997,
       -0.083642  ,  0.66324002, -0.45989001,  0.95883   ,  1.31850004,
       -0.62523001,  0.56152999,  0.65288001, -1.23640001,  0.29692   ,
        1.23800004,  0.80620998,  0.40967   ,  0.39945   ,  0.57982999,
       -1.36580002, -1.67639995,  0.069718  , -0.59634   , -0.63116997,
        2.58940005, -0.56914997,  0.46878999, -0.9249    , -0.44628999,
        0.42282999,  0.17636999, -0.014459  , -0.75357997, -0.97850001,
        0.023075  ,  1.2227    , -0.17517   , -1.57430005, -0.95349997,
       -0.10959   ,  0.24517   , -0.93550998, -0.20942999,  0.67333001])

combine tfidf with glove word embedding

In [23]:
word_idx = 1
for word in tfidf_dict:
  multipler = tfidf_dict[word]
  emb_matrix[word_idx] = emb_matrix[word_idx] * multipler
  word_idx = word_idx + 1
emb_matrix[1]

array([ 0.10478676,  0.33593313, -1.0098612 , -0.21316646,  0.09606211,
        0.63288962, -0.32932334, -1.57680423,  0.03952092,  2.02872745,
       -0.12680229,  1.00547994, -0.69719885,  1.45359795,  1.99886212,
       -0.94785632,  0.85128631,  0.98977405, -1.87439747,  0.45013434,
        1.87682313,  1.22222415,  0.6210647 ,  0.60557107,  0.87902933,
       -2.07056947, -2.54144273,  0.10569334, -0.9040587 , -0.95686137,
        3.92556201, -0.86283829,  0.71069134, -1.40215966, -0.67658105,
        0.64101541,  0.26737906, -0.02192002, -1.14243642, -1.48341793,
        0.03498198,  1.85362809, -0.26555986, -2.38665805, -1.44551757,
       -0.16613978,  0.3716807 , -1.41824452, -0.31749842,  1.02077649])

In [ ]:
print(vocab_len)

87516


In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_words, weights=[emb_matrix], trainable=True)

embedding_size=32
model=Sequential()
model.add(embedding_layer)
model.add(GRU(units=50, return_sequences = True, input_shape=(X_train.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dropout(0.6))
model.add(GRU(units=50, input_shape=(X_train.shape[1],1), activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 50)           4375800   
                                                                 
 gru (GRU)                   (None, 500, 50)           15300     
                                                                 
 dropout (Dropout)           (None, 500, 50)           0         
                                                                 
 gru_1 (GRU)                 (None, 50)                15300     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 4,406,451
Trainable params: 4,406,451
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
547/547 [==============================] - 143s 256ms/step - loss: 0.5531 - accuracy: 0.7093
Epoch 2/5
547/547 [==============================] - 141s 258ms/step - loss: 0.3722 - accuracy: 0.8387
Epoch 3/5
547/547 [==============================] - 139s 255ms/step - loss: 0.2929 - accuracy: 0.8819
Epoch 4/5
547/547 [==============================] - 139s 255ms/step - loss: 0.2410 - accuracy: 0.9027
Epoch 5/5
547/547 [==============================] - 139s 254ms/step - loss: 0.1939 - accuracy: 0.9273


In [ ]:
scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.8741333484649658


In [ ]:
model.save('basic_model')

INFO:tensorflow:Assets written to: basic_gru_model\assets


INFO:tensorflow:Assets written to: basic_gru_model\assets


In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_words, weights=[emb_matrix], trainable=True)

embedding_size=32
model=Sequential()
model.add(embedding_layer)
model.add(GRU(units=50, input_shape=(X_train.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 50)           4375800   
                                                                 
 gru_2 (GRU)                 (None, 50)                15300     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 4,391,151
Trainable params: 4,391,151
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
547/547 [==============================] - 77s 139ms/step - loss: 0.5690 - accuracy: 0.6942
Epoch 2/5
547/547 [==============================] - 75s 137ms/step - loss: 0.3971 - accuracy: 0.8254
Epoch 3/5
547/547 [==============================] - 75s 138ms/step - loss: 0.3005 - accuracy: 0.87

In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_words, weights=[emb_matrix], trainable=False)

embedding_size=32
model=Sequential()
model.add(embedding_layer)
model.add(GRU(units=50, input_shape=(X_train.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 50)           4375800   
                                                                 
 gru_3 (GRU)                 (None, 50)                15300     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 4,391,151
Trainable params: 15,351
Non-trainable params: 4,375,800
_________________________________________________________________
None
Epoch 1/5
547/547 [==============================] - 60s 108ms/step - loss: 0.5861 - accuracy: 0.6813
Epoch 2/5
547/547 [==============================] - 59s 108ms/step - loss: 0.4507 - accuracy: 0.7907
Epoch 3/5
547/547 [==============================] - 59s 107ms/step - loss: 0.3912 - accuracy:

# Using Test Data

In [24]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_words, weights=[emb_matrix], trainable=True)

embedding_size=32
model=Sequential()
model.add(embedding_layer)
model.add(GRU(units=50, return_sequences = True, input_shape=(dataset.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dropout(0.6))
model.add(GRU(units=50, input_shape=(dataset.shape[1],1), activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 50)           4375800   
                                                                 
 gru (GRU)                   (None, 500, 50)           15300     
                                                                 
 dropout (Dropout)           (None, 500, 50)           0         
                                                                 
 gru_1 (GRU)                 (None, 50)                15300     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 4,406,451
Trainable params: 4,406,451
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
batch_size=32
model.fit(dataset, y, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
782/782 [==============================] - 198s 251ms/step - loss: 0.5100 - accuracy: 0.7396
Epoch 2/5
782/782 [==============================] - 197s 253ms/step - loss: 0.3307 - accuracy: 0.8592
Epoch 3/5
782/782 [==============================] - 201s 257ms/step - loss: 0.2626 - accuracy: 0.8940
Epoch 4/5
782/782 [==============================] - 214s 274ms/step - loss: 0.2052 - accuracy: 0.9203
Epoch 5/5
782/782 [==============================] - 206s 264ms/step - loss: 0.1545 - accuracy: 0.9432


In [ ]:
test_df

,id,text,score
0,0,Once again Mr. Costner has dragged out a movie...,0
1,1,This is a pale imitation of 'Officer and a Gen...,0
2,2,"It seems ever since 1982, about every two or t...",0
3,3,"Wow, another Kevin Costner hero movie. Postman...",0
4,4,"Alas, another Costner movie that was an hour t...",0
...,...,...,...
24995,24995,"Sex, drugs, racism and of course you ABC's. Wh...",1
24996,24996,I love this show. My girlfriend was gonna get ...,1
24997,24997,"Seeing this show gives me respect for MTV, tho...",1
24998,24998,I'm torn about this show. While MOST parts of ...,1


In [26]:
#test_x = test_df['text']
test_df['tokens'] = test_df['text'].apply(tokenizeReview)
test_df['lemmTokens'] = test_df['tokens'].apply(lemmatizer)
test_df['lemmTokens'] = test_df['lemmTokens'].apply(remove_stopwords)
test_df['vecLem'] = test_df['lemmTokens'].apply(convertToIndex)
test_x = sequence.pad_sequences(test_df['vecLem'], maxlen=max_words)
test_x

# test_y = test_df['score']
# scores = model.evaluate(test_x, test_y, verbose=0)
# print('Test accuracy:', scores[1])

array([[   0,    0,    0, ...,  116,   86,  456],
       [   0,    0,    0, ...,    7,  182,  187],
       [   0,    0,    0, ...,   43, 6449,  309],
       ...,
       [   0,    0,    0, ...,  568,  639, 2133],
       [   0,    0,    0, ...,   14,  276,    8],
       [   0,    0,    0, ...,  885, 2600, 2013]])

In [27]:
test_y = test_df['score']
scores = model.evaluate(test_x, test_y, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.8631200194358826


In [28]:
pred = model.predict(test_x)

In [29]:
from sklearn.metrics import accuracy_score
pred_labels = []
for i in pred:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_y,pred_labels))

Accuracy of prediction on test set :  0.86312


In [30]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(test_y,pred_labels)
print(cm)

[[11590   910]
 [ 2512  9988]]


In [31]:
test_df['pred'] = pred_labels
test_df
test_df.to_csv("test_df_pred_lemma_stopword__tfidfv2_trainable_true_gru.csv")

Trainable false

In [32]:
embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_words, weights=[emb_matrix], trainable=False)

embedding_size=32
model=Sequential()
model.add(embedding_layer)
model.add(GRU(units=50, return_sequences = True, input_shape=(dataset.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dropout(0.6))
model.add(GRU(units=50, input_shape=(dataset.shape[1],1), activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 50)           4375800   
                                                                 
 gru_2 (GRU)                 (None, 500, 50)           15300     
                                                                 
 dropout_1 (Dropout)         (None, 500, 50)           0         
                                                                 
 gru_3 (GRU)                 (None, 50)                15300     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 4,406,451
Trainable params: 30,651
Non-trainable params: 4,375,800
_________________________________________________________________
None


In [33]:
model.fit(dataset, y, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
782/782 [==============================] - 179s 226ms/step - loss: 0.5173 - accuracy: 0.7388
Epoch 2/5
782/782 [==============================] - 173s 221ms/step - loss: 0.3801 - accuracy: 0.8337
Epoch 3/5
782/782 [==============================] - 172s 220ms/step - loss: 0.3425 - accuracy: 0.8510
Epoch 4/5
782/782 [==============================] - 173s 221ms/step - loss: 0.3126 - accuracy: 0.8692
Epoch 5/5
782/782 [==============================] - 172s 221ms/step - loss: 0.2956 - accuracy: 0.8761


In [34]:
scores = model.evaluate(test_x, test_y, verbose=0)
print('Test accuracy:', scores[1])
pred = model.predict(test_x)

pred_labels = []
for i in pred:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_y,pred_labels))

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(test_y,pred_labels)
print(cm)

test_df['pred'] = pred_labels
test_df
test_df.to_csv("test_df_pred_lemma_stopword__tfidfv2_trainable_falsey_gru.csv")

Test accuracy: 0.8627200126647949
Accuracy of prediction on test set :  0.86272
[[10890  1610]
 [ 1822 10678]]
